In [ ]:
import requests
import pandas as pd
from datetime import datetime
import numpy as np
from astropy.coordinates import SkyCoord
import astropy.units as u
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import pickle
import scipy.stats
from matplotlib import cm
import matplotlib as mpl
from pandas.io.html import read_html
from selenium import webdriver
import contextlib
import csv, smtplib, ssl
import time



def checkObs():
    output=[]
    
    #list of target names to be observed
    target_file=open('xmm_targets.csv')
    target_names=[]
    for row in target_file:
        target_names=np.append(target_names,row)

    url = 'https://xmm-tools.cosmos.esa.int/external/xmm_sched/short_term_schedule.php'
    html = requests.get(url).content
    df_list = pd.read_html(html,header =0)
    df = pd.DataFrame(df_list[-1])

    for ind,name in enumerate(df["Target Name"]):
        name=''.join(name.split()) #remove spaces in source names
        for target_name in target_names:
            target_name=''.join(target_name.split()) #remove spaces in target names just to be sure
            if target_name in name:
                output=np.append(output,[target_name,df["UTC Obs Startyyyy-mm-dd hh:mm:ss"][ind],df["UTC Obs Endyyyy-mm-dd hh:mm:ss"][ind],df["PI"][ind]])
    return output

def sendEmail(password="IceCube!3",message="Testmail"):
    port = 587
    smtp_server="smtp.mail.de"
    sender_email="neutrino.alert@mail.de"
    
    context=ssl.create_default_context()
    
    with smtplib.SMTP(smtp_server, port) as server:
        server.starttls(context=context)
        server.login("neutrino.alert@mail.de",password)
        print("sending email")
        server.sendmail("neutrino.alert@mail.de","florian.eppel@7fun.de",message)

while True:
    data=checkObs()
    #generate message to send
    rows=[]
    for j in range(int(len(data)/4)):
        row=""
        for i in range(4):
            row=row+data[i+j*4]+" "
        rows=np.append(rows,row)
    
    output_string=""
    for row in rows:
        output_string=output_string+row+"\n"
    
    
    output_string=" Hi Flo,\n"+"Here is a list of upcoming XMM-Newton Targets to shadow search:\n"+output_string
    
    sendEmail(message=output_string)
    
    time.sleep(86400)#sleep for one day